# 기존 오픈 마켓은 제품 데이터 어떻게 가져오지

- 다나와 기술 블로그 - [다나와의의 상품 데이터 여정](https://danawalab.github.io/common/2023/08/30/Danawa-Data.html)

# Youtube Data API를 활용해보자
원래는 beautifulsoup으로 긁어오려고 했는데
나중에 서비스 데이터 수집에 자동화까지 생각하면 법적으로 문제가 생길 수도 있겠다는 생각

Google Cloud Platform에 API 찾아보니 유튜브 데이터 API도 있길래 이걸 하기로 결정

그래서 찾은 참고 페이지↓↓↓

- 공식 문서 : [YouTube Data API](https://developers.google.com/youtube/v3/getting-started?hl=ko )
- 참고 문서 
    - [YouTube API 환경설정](https://velog.io/@ssongji/%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%81%AC%EB%A1%A4%EB%A7%81-%EB%B0%8F-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-1.-YOUTUBE-API-%EC%82%AC%EC%9A%A9-%ED%99%98%EA%B2%BD-%EC%84%A4%EC%A0%95)
    - [YouTube 상의 데이터 크롤링 실습](https://velog.io/@ssongji/%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%81%AC%EB%A1%A4%EB%A7%81-%EB%B0%8F-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-2.-YOUTUBE-API%EB%A1%9C-%ED%82%A4%EC%9B%8C%EB%93%9C-%EA%B2%80%EC%83%89-%EB%B0%8F-%ED%8A%B9%EC%A0%95-%EC%9C%A0%ED%8A%9C%EB%B2%84-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%B6%94%EC%B6%9C)

In [1]:
# 설치 안 되어 있으면 설치하기 
!pip install --upgrade google-api-python-client
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 36.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 14.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 10.1 MB/s eta 0:0

In [1]:
# 필요 라이브러리
import pandas as pd
import re
import requests


from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
# from oauth2client.tools import argparser # Oauth 필요할 때만 쓰자

from env import YOUTUBE_API

In [393]:
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_SERVICE_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_SERVICE_VERSION, developerKey = YOUTUBE_API)

## 1️⃣ 유튜브 데이터 뽑아오기

In [394]:
# 검색어 정의
search_keyword = ["#선물추천", "선물언박싱", "생일선물언박싱", "#카카오톡선물하기"]

In [395]:
# 영상에서 정보 뽑아오는 함수
# 스크립트 정보 제외

def getVideoInfo(searchQ, maxCount):
    # 검색 키워드로 search한 결과 최신순으로 받아오기
    search_response = youtube.search().list(
        order = 'date',
        part = 'snippet',
        q = searchQ,
        maxResults = maxCount,
    ).execute()

    # videoId 리스트 따로 빼서 video 정보를 가져와 보자
    video_ids = []
    for i in range(0, len(search_response['items'])):
        video_ids.append((search_response['items'][i]['id']['videoId'])) #videoId의 리스트를 만들어 둔다 (videoId로 조회할 수 있게)

    #추출할 정보의 list들과 그 모든 정보를 key-value로 저장할 딕셔너리 변수를 하나 생성한다.
    videoId = []
    pubDates= []
    videoTitle = []
    videoDescription = []
    channelTitle = []
    videoTags = []
    videoLive = []
    videoType = []
    videoViewcount = []
    data_dicts = { }


    for i in range(0, len(search_response['items'])):
        video_info = youtube.videos().list(
            part='snippet, statistics, topicDetails, contentDetails, player',
            id=video_ids[i],
        ).execute()

        temp_videoId = video_info['items'][0]['id']
        temp_pubDates = video_info['items'][0]['snippet'].get('publishedAt') # 나중엔 ~~일 이전의 데이터는 제거
        temp_videoTitle = video_info['items'][0]['snippet'].get('title')
        temp_videoDescription = video_info['items'][0]['snippet'].get('description') # None? 처리
        if temp_videoDescription == '' or temp_videoDescription == 'none' or temp_videoDescription is None: continue
        temp_channelTitle = video_info['items'][0]['snippet'].get('channelTitle') 
        temp_videoTags =  video_info['items'][0]['snippet'].get('tags') # None? 처리
        if temp_videoTags is None or temp_videoTags=='none' : temp_videoTags = []
        temp_videoLive = video_info['items'][0]['snippet'].get('liveBroadcastContent') #'none' 이 아니라면 제외하기
        if temp_videoLive != 'none': continue
        temp_videoType = video_info['items'][0]['contentDetails'].get('duration')
        if len(temp_videoType) == 5: continue
        temp_videoViewcount = video_info['items'][0]['statistics'].get('viewCount')
        if temp_videoViewcount is None:
            temp_videoViewcount = "0"

        videoId.append(temp_videoId)
        pubDates.append(temp_pubDates)
        videoTitle.append(temp_videoTitle)
        videoDescription.append(temp_videoDescription)
        channelTitle.append(temp_channelTitle)
        videoTags.append(temp_videoTags)
        videoType.append(temp_videoType)
        videoViewcount.append(temp_videoViewcount)
        
    data_dicts['id'] = videoId
    data_dicts['date'] = pubDates
    data_dicts['title'] = videoTitle
    data_dicts['description'] = videoDescription
    data_dicts['channel'] = channelTitle
    data_dicts['tags'] = videoTags
    data_dicts['type'] = videoType
    data_dicts['view'] = videoViewcount 
    
    return data_dicts

In [ ]:
for key in search_keyword:
    data = getVideoInfo(key, 50)
    df = pd.DataFrame.from_dict(data=data)
    df.to_csv('./'+key+'.csv')

In [ ]:
# 데이터셋 합치기

total_dataset = pd.DataFrame(columns=['searchQ', 'id', 'date', 'title', 'description', 'channel', 'tags', 'type', 'view'])

for key in search_keyword:
    df = pd.read_csv('./'+key+'.csv')
    df['searchQ'] = [key] * len(df)
    df = df[['searchQ', 'id', 'date', 'title', 'description', 'channel', 'tags', 'type', 'view']]
    total_dataset = pd.concat([total_dataset, df],axis=0)
total_dataset.reset_index(inplace=True, drop=True)
total_dataset

,searchQ,id,date,title,description,channel,tags,type,view
0,#선물추천,mLDpy3Pv_XU,2024-05-01T11:00:43Z,💫1만원~7만원대 갓성비 꿀템 | 요즘 센스있는 선물추천🎁 | 가격대별 뷰티/푸드 ...,#선물추천 #선물하기 #29CM\n*위 영상은 29CM의 유료광고를 포함하고 있습니...,꼬민지,"['선물추천', '선물추천5만원', '집들이선물', '카카오톡 선물하기', '생일선...",PT12M4S,30548
1,#선물추천,9CqqIRMlBgE,2024-02-03T15:12:38Z,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ...",* 이 영상은 유료광고를 포함하지 않습니다.\n\n\n\n⏱ Timeline\n00...,Minjeong Park,"['부담없는선물', '여자선물', '남자선물', '집들이선물', '20대선물', '...",PT16M20S,36751
2,#선물추천,aeGJi2iuavc,2024-02-02T01:00:01Z,#kbeauty #에바끌레르골프전용화장품 #에바끌레르 #선물추천 #기능성화장품 #에...,#에바끌레르 골프전용 화장품 #골프화장품#골퍼화장품\n #kbeauty #골프장 #...,에바끌레르 골프전용화장품TV,[],PT4M40S,2128
3,#선물추천,fflimV2pIL8,2023-12-12T12:12:40Z,"선물 사기전에 꼭 보세요❗️뻔하지 않은 선물 20가지 가져옴🎁 생일, 크리스마스, ...",탤꾸들 안녕하세요\n선물로 고민인 탤꾸들 영상 보시고 도움 되었으면 좋겠어요!\n\...,태리태리 taeritaeri,"['강태리', '뷰티', '브이로그', '태리태리', '패션', '여자패션', '여...",PT13M43S,75279
4,#선물추천,ONFJdkVFSzY,2023-12-04T10:54:31Z,"5만원 미만 역대급 반응 선물 추천💝 (카카오톡 선물하기, 1인가구, 생일, 집들이...",✉️ instagram : @mixmansion \n빠른 데일리룩을 구경하러 오세요...,mixmansion 혜진,[],PT12M35S,78017
...,...,...,...,...,...,...,...,...,...
98,#카카오톡선물하기,96jUU0dJ92w,2023-02-22T08:30:03Z,광고❌ 내돈내선🎁 카카오톡 선물하기 찐추천템 (랭킹에선 못찾음 주의),광고❌ 내돈내선🎁 #카카오톡 선물하기 찐추천템 (랭킹에선 못찾음 주의) #선물 #선...,스리슬쩍,"['카카오톡', '카카오톡선물하기', '선물', '생일선물', '발렌타인데이', '...",PT18M7S,10602
99,#카카오톡선물하기,kOky8fquvtE,2023-01-22T13:00:28Z,🎁카카오톡 선물하기에서 '이거' 사주면 무조건 좋아한다....❤️? #생일선물추천,#카카오톡선물하기 #생일선물 #친구선물 #지인선물\n-----------------...,10시엔 디붕,"['벨킨마사지건', '미니화로대', '카톡선물', '카카오톡선물추천', '선물추천템...",PT11M29S,19077
100,#카카오톡선물하기,7nbWqtOCisI,2022-09-11T09:00:07Z,광고x)생일선물 와랄라 언박싱🎁🛍카톡선물하기 선물추천!,이 영광을 어모니에게 바칩니다\n\n\n인스타그램 / Instagram\njina....,보고싶진아,"['진아', '보고싶진아', '리플', '일상', '브이로그', 'vlog', '메...",PT14M46S,40475
101,#카카오톡선물하기,JIY9HNGJ_fY,2022-09-01T13:45:00Z,무조건 성공하는 카톡 선물하기템ㅣ프로선물러의 추석 선물 추천🎁,안녕하세요! 세이영이에요😆\n\n\n\n이번 주만 지나면 다음 주부터는 바로바로 추...,세이영Sayyoung,[],PT14M21S,3025


### 유튜브 영상 컨텐츠 필터링

#### 1. 중복 영상 제거

In [ ]:
# 겹치는 영상 개수 체크
duplicates = total_dataset.duplicated('id')
duplicates = duplicates[duplicates==True]
len(duplicates)

16

In [ ]:
total_dataset.drop_duplicates(subset='id', keep='first',inplace=True)
total_dataset.reset_index(inplace=True,drop=True)

total_dataset.to_csv('./youtube_dataset.csv')

In [ ]:
total_dataset = pd.read_csv('./youtube_dataset.csv', index_col=0)
total_dataset

,searchQ,id,date,title,description,channel,tags,type,view
0,#선물추천,mLDpy3Pv_XU,2024-05-01T11:00:43Z,💫1만원~7만원대 갓성비 꿀템 | 요즘 센스있는 선물추천🎁 | 가격대별 뷰티/푸드 ...,#선물추천 #선물하기 #29CM\n*위 영상은 29CM의 유료광고를 포함하고 있습니...,꼬민지,"['선물추천', '선물추천5만원', '집들이선물', '카카오톡 선물하기', '생일선...",PT12M4S,30548
1,#선물추천,9CqqIRMlBgE,2024-02-03T15:12:38Z,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ...",* 이 영상은 유료광고를 포함하지 않습니다.\n\n\n\n⏱ Timeline\n00...,Minjeong Park,"['부담없는선물', '여자선물', '남자선물', '집들이선물', '20대선물', '...",PT16M20S,36751
2,#선물추천,aeGJi2iuavc,2024-02-02T01:00:01Z,#kbeauty #에바끌레르골프전용화장품 #에바끌레르 #선물추천 #기능성화장품 #에...,#에바끌레르 골프전용 화장품 #골프화장품#골퍼화장품\n #kbeauty #골프장 #...,에바끌레르 골프전용화장품TV,[],PT4M40S,2128
3,#선물추천,fflimV2pIL8,2023-12-12T12:12:40Z,"선물 사기전에 꼭 보세요❗️뻔하지 않은 선물 20가지 가져옴🎁 생일, 크리스마스, ...",탤꾸들 안녕하세요\n선물로 고민인 탤꾸들 영상 보시고 도움 되었으면 좋겠어요!\n\...,태리태리 taeritaeri,"['강태리', '뷰티', '브이로그', '태리태리', '패션', '여자패션', '여...",PT13M43S,75279
4,#선물추천,ONFJdkVFSzY,2023-12-04T10:54:31Z,"5만원 미만 역대급 반응 선물 추천💝 (카카오톡 선물하기, 1인가구, 생일, 집들이...",✉️ instagram : @mixmansion \n빠른 데일리룩을 구경하러 오세요...,mixmansion 혜진,[],PT12M35S,78017
...,...,...,...,...,...,...,...,...,...
82,#카카오톡선물하기,96jUU0dJ92w,2023-02-22T08:30:03Z,광고❌ 내돈내선🎁 카카오톡 선물하기 찐추천템 (랭킹에선 못찾음 주의),광고❌ 내돈내선🎁 #카카오톡 선물하기 찐추천템 (랭킹에선 못찾음 주의) #선물 #선...,스리슬쩍,"['카카오톡', '카카오톡선물하기', '선물', '생일선물', '발렌타인데이', '...",PT18M7S,10602
83,#카카오톡선물하기,kOky8fquvtE,2023-01-22T13:00:28Z,🎁카카오톡 선물하기에서 '이거' 사주면 무조건 좋아한다....❤️? #생일선물추천,#카카오톡선물하기 #생일선물 #친구선물 #지인선물\n-----------------...,10시엔 디붕,"['벨킨마사지건', '미니화로대', '카톡선물', '카카오톡선물추천', '선물추천템...",PT11M29S,19077
84,#카카오톡선물하기,7nbWqtOCisI,2022-09-11T09:00:07Z,광고x)생일선물 와랄라 언박싱🎁🛍카톡선물하기 선물추천!,이 영광을 어모니에게 바칩니다\n\n\n인스타그램 / Instagram\njina....,보고싶진아,"['진아', '보고싶진아', '리플', '일상', '브이로그', 'vlog', '메...",PT14M46S,40475
85,#카카오톡선물하기,JIY9HNGJ_fY,2022-09-01T13:45:00Z,무조건 성공하는 카톡 선물하기템ㅣ프로선물러의 추석 선물 추천🎁,안녕하세요! 세이영이에요😆\n\n\n\n이번 주만 지나면 다음 주부터는 바로바로 추...,세이영Sayyoung,[],PT14M21S,3025


#### 2. deiscription에 제품 정보가 없는 컨텐츠 제거

In [ ]:
drop_idx = []
for i in range(len(total_dataset)):
    if '\n' not in total_dataset['description'][i]:
        print(i, total_dataset['description'][i])
        drop_idx.append(i)
    line_txts = total_dataset['description'][i].split('\n')
    line_txts = list(filter(None, line_txts))
    if len(line_txts) <10:
        print(i, total_dataset['description'][i])
        drop_idx.append(i)


2 #에바끌레르 골프전용 화장품 #골프화장품#골퍼화장품
 #kbeauty #골프장 #선물추천 #선물추천 #외국인반응
#내인생멋지게 살고싶ㄷㅏ #외국인들 추천에바끌레르 #기능성화장품 #프로골프추천 #호델 #피부과추천 #성형상담 에바끌레르 화장품 #베트남 인기 #외국인 좋아하는 선크림 #중국골프추천 #모델추천 에바끌레르 #k뷰티 #골프화장품 #evaclairemail ❤🧡💛💚💙💜
10 안녕하세요 여러분! 오늘은 10만원 이하 가격대별로 센스있고 고급스러운 선물들을 공유하려고 합니다. 여러분의 선물 아이디어도 댓글로 공유해주세요~ :)

💙인테리언니 블로그 바로가기
https://m.blog.naver.com/tearsineye

📩섭외 및 기타 제휴 문의
intelliunnie@outlook.com

#명품선물 #생일선물 #선물추천
30 진's Greetings 의 석지니 선물 언박싱!!!
석진이가 디자인한 알람시계... 아끼지말고
사용합시다! 아미~~😁 사랑스런남자
#jin #sekjin #석진 #2024btsfesta #bts #bts #btsarmy #kpop #armylog #army #unboxing #rm #jin #suga #jhope #jimin #V #btsv #jk #페스타
31 #굿즈 #굿즈깡 #고전문구 #스누피 #지브리 #덕질 #디즈니 #럭키박스 #산리오 #리틀트윈스타 #미녀와야수 #덕후룸 #선물언박싱 #택배개봉기 #택배맛집
31 #굿즈 #굿즈깡 #고전문구 #스누피 #지브리 #덕질 #디즈니 #럭키박스 #산리오 #리틀트윈스타 #미녀와야수 #덕후룸 #선물언박싱 #택배개봉기 #택배맛집
32 #생일선물언박싱 #생일선물추천 #카카오톡선물하기
32 #생일선물언박싱 #생일선물추천 #카카오톡선물하기
34 This stream is created with #PRISMLiveStudio
34 This stream is created with #PRISMLiveStudio
35 친구가 준 선물 개봉
35 친구가 준 선물 개봉
36 에르메스 산토리니 샌들 사이즈 

In [ ]:
total_dataset = total_dataset.drop(labels=drop_idx)
total_dataset.reset_index(drop=True, inplace=True)
len(total_dataset)

57

In [ ]:
total_dataset.to_csv('./youtube_dataset_goodDescriptions.csv')

In [ ]:
total_dataset = pd.read_csv('./youtube_dataset_goodDescriptions.csv', index_col=0)

In [ ]:
total_dataset.sample(10)

,searchQ,id,date,title,description,channel,tags,type,view
14,#선물추천,YfmkmsMRYKs,2023-03-24T09:15:01Z,"🎁선물추천, 이거 다 드림 🎁｜3040 취향 저격하는 10만원 이하 센스있는 선물(...",* 본 영상은 록시땅의 유료 광고를 포함하고 있습니다*\n\n✔️구매 링크 : ht...,한혜진 Han Hye Jin,"['한혜진', '모델 한혜진', 'han hyejin', 'han hye jin',...",PT14M45S,685390
52,#카카오톡선물하기,SDOVtmEqVGE,2023-05-02T10:20:51Z,"1~10만원대 🎁센스있는 카카오톡 선물하기 추천템 (뉴진스st 카메라 ,부모님 선물...","#카카오톡선물하기 #카톡선물 #선물추천\n\n'슬밋, VT코스메틱, LG Pra.L...",제이블릿 Jblit,"['카카오톡', '카카오톡 선물하기', '카카오톡 선물하기 여자친구', '카카오톡 ...",PT7M33S,1223
16,#선물추천,cMcpE-3re8c,2023-03-08T09:00:32Z,[선물추천]여자는 이런 선물 받으면 100%좋아해요❣️ 이 영상으로 선물고민 끝!!...,"반가워요 여러분🤗💜\n선물로 받고 싶은 게 있고, 선물을 줬을 때\n반응이 너무 좋...",Bora Claire (보라끌레르),"['패션', '패션유튜버', '스타일링', '패션디자이너', '룩북', '데일리룩'...",PT12M52S,343662
9,#선물추천,b0m2sfoBO8g,2023-06-25T03:00:26Z,10만원 이하 고급 선물 추천🛍명품 신상,#선물추천 #샤넬 #디올 #에르메스 #명품\n\n로율님들~ 💕 10만원 이하 선물추...,이율아트 Leeyull Art,"['선물추천', '선물 추천', '10만원 이하 선물추천', '5만원 이하 선물 추...",PT22M48S,40336
46,#카카오톡선물하기,L70D37zEOII,2023-09-24T12:00:24Z,선물 고민 NO! 🎁 요즘 핫한 2-3만원대 카카오톡 선물하기 ‘패션’ 아이템 추천...,#카카오톡선물하기 #2-3만원대선물 #구독자이벤트 \n\n\n** 이 영상은 밀로의...,제제로 JEJELO,"['구독자이벤트', '카카오톡선물하기', '2-3만원대선물추천', '데일리룩', '...",PT8M55S,6160
12,#선물추천,gu_JBO6jmIo,2023-04-30T11:49:00Z,흔하지 않고 센스있는 선물 추천 (5만원 이하),#선물추천 #선물추천5만원 #5만원대선물\n제 채널에서 나름 높은 뷰를 자랑하는 선...,관리하는 남자 아우라M,"['선물추천템', '5만원대 선물추천', '5만원 미만 선물추천', '5만원 이사 ...",PT9M47S,44312
43,#카카오톡선물하기,1d35prQCTKA,2023-12-30T10:00:02Z,"기프티콘 그만❌센스있는 카톡 선물하기 추천템🎁 | 1만원~5만원대 생일 선물 추천,...",좋아요와 구독은 영상 제작에 큰 힘이 됩니다🤍\n\n𝗜𝗻𝘀𝘁𝗮𝗴𝗿𝗮𝗺 • https...,프롬하정 hajeong,"['데일리메이크업', '여쿨메이크업', '여쿨라', '여쿨라이트', '여쿨라이트메이...",PT14M10S,2048
51,#카카오톡선물하기,JWRlTxyS0c8,2023-05-04T10:00:09Z,얘..센스터진다?.. 카톡 선물하기 추천템🎁1만원대~10만원대 흔하지 않은 유니크+...,"'슬밋, VT코스메틱, LG Pra.L, 토코보의 유료 광고를 포함하고 있으며 스튜...",daaawon다원,"['카톡선물하기', '어버이날선물', '집들이선물', '올리브영추천템']",PT12M20S,2397
26,#선물추천,D9_PWsoMXXI,2021-06-18T09:00:11Z,10만원으로 에르메스 선물하기🎁 가격대별 센스있는 선물 추천 (※내돈내산) 【펄이지엥】,펄이피플! 한 주간 잘 보내셨어요? ㅎㅎ\n\n오늘은 직장 후배들이 종종 했던 고민...,펄이지엥 Pearlysien,"['브이로그', 'vlog', '펄이지엥', '조현주', '현주언니', '디자이너'...",PT15M56S,1510953
19,#선물추천,gX4F9weydsI,2022-11-24T12:14:54Z,[ENG/JPN] 가격대별 센스있는 여자 선물 추천🎁(3만원~30만원대) | 상황별...,📢이 영상은 온호프의 부분유료광고를 포함하고 있습니다.\n📢본 채널의 모든 콘텐츠 ...,재유JEYU,"['선물추천', '생일선물 추천', '생일 선물', '기념일 선물', '선물', '...",PT22M17S,82242


## 2️⃣ Description에서 제품 정보만 추출해보자

### Description Data 뜯어보기

#### 1. 지울만한 데이터 기준 값 (예상)

**✅ 제품 정보가 아닐 확률이 높은 텍스트**
- 태그 모두 지우기(\#000형태)
- 블로그, 인스타, 제휴와 같은 단어가 들어간 줄의 데이터

**✅ 제품 정보일 확률이 높은 텍스트**
- 번호로 시작하는 줄의 정보
- 번호로 시작하는 줄 바로 밑에 있는 url일 경우
- 금액과 관련한 문자가 포함되어있을 경우

**✅ 추천하기에 괜찮은 제품 리스트가 없는 영상 기준**
- 제품 정보가 하나도 없는 영상(이미 거의 대체로 필터링 완료)
- description에 이모지가 하나도 포함되지 않은 경우(트렌디함↓)
- 채널 관련(본인 브랜드 홍보용 또는 트렌디하지 않은 채널)
    - 채널 이름이 00TV 이런 식일 때
    - 채널 구독자 수? 10만 언더


#### 2. 제품 정보 기준 값 (결론)

**✅ 토글과 함께 링크가 있는 데이터**
- 토글로 예상되는 문자가 있고
- url이 거기에 근접하게 위치함(같은 줄 또는 바로 아래 줄)

**✅ 토글과 함께 금액 텍스트가 있는 데이터**
- 토글로 예상되는 문자가 있고
- 숫자 + 원 / 숫자3개 + , + 숫자3개 이런 텍스트가 함께 있는 경우


In [3]:
total_dataset = pd.read_csv('./youtube_dataset_goodDescriptions.csv', index_col=0)

In [4]:
## 데이터 보려면 샘플 출력해야함
sample = total_dataset.sample(1).reset_index(drop=True)
print(sample['description'][0])

#카카오톡선물하기 #생일선물 #친구선물 #지인선물
---------------------------------------------------
📢 이 영상은 '니몸내몸'의 부분 유료광고가 포함되어 있습니다.

-니몸내몸 영양젤리 바로보기!🧼💙
 http://bit.ly/3GFZhya

---------------------------------------------------
 ‼️ 니몸내몸 영양젤리 할인 받는 법 ‼️

*적용 상품 주문시 할인코드 적용란에 “buung” 입력 ➡️ 적용 
(적용 부분 : 집중젤리 세트, 집중젤리 단품, 영양젤리 3종) 

✔️할인코드 : buung
- 할인코드 발급은 공식몰 회원만 가능하며, 
할인코드 적용은 ID당 최대 1회 사용만 가능합니다.

- 기간 : 2023-01-27(금) 23:59까지 (기간 꼭 확인해 주세요!)

*반드시 결제 전 할인쿠폰 프로모션 코드 및 코드 적용 여부 확인하시길 바랍니다.
할인코드는 타 쿠폰, 적립금과 동시사용 불가합니다.

---------------------------------------------------

🎁 구독자 이벤트 🎁

이벤트 폼 : https://forms.gle/PTRiJ6E2mXPkpEMH8

- 추첨을 통해 구독자 3️⃣분께
1)니몸내몸 영양젤리 3종 set (2명)
2)니몸내몸 집중젤리 2종 set(1명)를 드립니다!

- 구독자 이벤트 기간: 2023-01-26(목) 23:59까지

- 구독자 이벤트 당첨자 발표: 2023-01-27(금) 22:00 

- 발표 장소: [10시엔 디붕] 커뮤니티


—————————————————————————
🔅 모든 문의는 instagram @b.uung
협찬&광고 문의는 b.uuuuuuuung@gmail.com 
—————————————————————————


### 일단 URL이 있는 데이터만 활용
#### 제품 URL을 찾아보자

In [35]:
product_info = []
for i in range(len(total_dataset)):
    content_info = total_dataset.loc[i]
    product = content_info['product_info']

    line_txts = product.split('\n')
    line_txts = list(filter(None, line_txts))


    for line in range(len(line_txts)):
        txt = line_txts[line]
        # if 'http' in txt and 'm.site' in txt: # 링크가 포함된 줄의 정보가 나옴
        #     print(i,line, txt)
        if 'brand' in txt: print(total_dataset['id'][i],line, txt)


fflimV2pIL8 38 https://product.29cm.co.kr/catalog/1967440?source=brand
kMHj6Zv4bW8 42 💛티파니앤코 https://gift.kakao.com/brand/11232
zHvq4WO9iI4 14 https://www.ssfshop.com/TEKLA/GM0022100536711/good?utag=$ref_prd:GM0022102097600$otherColor:D&dspCtgryNo=SFMB84A05A02&brandShopNo=&brndShopId=MCBR
zHvq4WO9iI4 37 https://brand.naver.com/sabre_korea
zHvq4WO9iI4 43 https://brand.naver.com/loivie/products/6829341185
zHvq4WO9iI4 45 https://brand.naver.com/loivie/products/7472273450


'www.instagram.com/zy00n_'

**제품 링크인 경우**
- 제품에 대한 단어 : product, gift, goods, catalog, promotion, item(약간 애매)
- https://bit.ly/ ~~ 이럼 들어가서 링크 다시 따와야함
- https://t.ly/ ~~ 이럼 들어가서 링크 다시 따와야함
- https://m.site.naver.com/ 이것도 다시 링크 떠오기
- 브랜드, 쇼핑몰 이름 : 29cm, oliveyoung, wconcept, kko.to, musinsa, gict.kakao.com, sivillage, smartstore, makers.kakao, kurly, nike


In [5]:
def check_url(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        if response.status_code == 200:
            return True
        else:
            # print(f"URL returned status code {response.status_code}.")
            return False
    except requests.exceptions.RequestException as e:
        # print(f"URL check failed: {e}")
        return False
    
def get_final_url(short_url):
    try:
        response = requests.head(short_url, allow_redirects=True, timeout=5)
        return response.url
    except requests.exceptions.RequestException as e:
        print(f"Failed to expand URL: {e}")
        return short_url

# def check_url(url):
#     try:
#         response = requests.head(url, allow_redirects=True, timeout=5)
#         if response.status_code == 200:
#             return (1, url)
#         else:
#             return False
#     except requests.exceptions.RequestException as e:
#         return False

In [ ]:
def extract_urls(text):
    # URL을 찾기 위한 정규 표현식 패턴
    url_pattern = re.compile(r'(https?://\S+)')
    
    # 패턴에 맞는 모든 URL을 찾기
    urls = url_pattern.findall(text)
    
    return urls

In [ ]:
# 제품 url 뽑기 (최적화 X)
product_url = []
yt_vID = []
product_key = ['product', 'gift', 'goods', 'catalog', 'promotion', 'item']
brand_key = ['29cm', 'oliveyoung', 'wconcept', 'kko.to', 'musinsa', 'gict.kakao.com', 'sivillage', 'smartstore', 'makers.kakao', 'kurly', 'nike']

for i in range(len(total_dataset)):
    content_info = total_dataset.loc[i]
    product_info = content_info['product_info']

    urls = extract_urls(product_info)

    for url in urls:
        # if ' ' in url : url = url.strip()
        
        # 유효하지 않은 url 확인
        if not check_url(url): continue
        # short url인 경우를 대비하여 최종 url 받아오기
        url = get_final_url(url)
        # 제품에 대한 단어가 있는 경우
        if any(word in url for word in product_key): 
            product_url.append(url)
            yt_vID.append(total_dataset['id'][i])
        # 브랜드에 대한 단어가 있는 경우
        elif any(word in url for word in brand_key): 
            product_url.append(url)
            yt_vID.append(total_dataset['id'][i])


In [32]:
productDF = pd.DataFrame([yt_vID, product_url]).T
productDF.columns =['videoID', 'productURL']
productDF.to_csv('./product.csv')

In [33]:
productDF = pd.read_csv("./product.csv",index_col=0)
productDF

,videoID,productURL
0,mLDpy3Pv_XU,https://content.29cm.co.kr/post/41215
1,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/1814100
2,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/2209304
3,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/1970379
4,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/1671349
...,...,...
117,JWRlTxyS0c8,https://gift.kakao.com/product/6983306
118,JWRlTxyS0c8,https://gift.kakao.com/product/4938088
119,SDOVtmEqVGE,https://gift.kakao.com/product/6780518?input_c...
120,SDOVtmEqVGE,https://makers.kakao.com/promotion/392


#### description에서 태그 제거하기

In [7]:
product_info = []
for i in range(len(total_dataset)):
    content_info = total_dataset.loc[i]
    description = content_info['description']
    pattern = '#([0-9a-zA-Z가-힣]*)'
    description = re.sub(pattern, '', description)
    description = re.sub('  +\n', '\n', description)

    product_info.append(description)

total_dataset['product_info'] = product_info

#### bullet을 찾아보자

In [8]:
product_info = []
for i in range(len(total_dataset)):
    content_info = total_dataset.loc[i]
    product = content_info['product_info']

    line_txts = product.split('\n')
    line_txts = list(filter(None, line_txts))
    line_first_txt = [line[0] for line in line_txts]

    might_be_bullet = []
    might_be_product_bullet = ''
    max_count = -1
    for t in list(set(line_first_txt)):
        count = line_first_txt.count(t)
        if count > 1 and not re.match('[a-zA-Z가-힣0-9]', t):
            might_be_bullet.append((t,count))
            if count > max_count:
                might_be_product_bullet = t
                max_count = count

    
    print(i, ' : ', might_be_product_bullet,max_count)

0  :  [ 6
1  :  ❥ 11
2  :  🎁 20
3  :   -1
4  :  💛 18
5  :   -1
6  :  * 7
7  :  🔔 2
8  :  * 2
9  :  🌳 5
10  :  ✷ 3
11  :  - 5
12  :    12
13  :   -1
14  :  ✔ 9
15  :  - 4
16  :  ✔ 16
17  :   -1
18  :  - 7
19  :  - 4
20  :  - 5
21  :  * 2
22  :   -1
23  :  ( 4
24  :   -1
25  :    3
26  :   -1
27  :  ✶ 4
28  :  ▶ 3
29  :   -1
30  :    20
31  :   -1
32  :  * 3
33  :    2
34  :   -1
35  :   -1
36  :  _ 2
37  :   -1
38  :   -1
39  :   -1
40  :  🎵 2
41  :  - 4
42  :  📍 24
43  :  🥚 2
44  :  ✔ 11
45  :  ( 2
 4  :  
47  :  - 5
48  :    3
49  :  ఇ 2
50  :  ❥ 10
51  :    2
52  :   -1
53  :   -1
54  :  - 10
55  :  🎁 2
56  :  . 5


In [ ]:
# bullet 후보를 찾아주는 함수
def find_bullet(text):
    line_txts = text.split('\n')
    line_txts = list(filter(None, line_txts))
    line_first_txt = [line[0] for line in line_txts]

    might_be_bullet = []
    might_be_product_bullet = ''
    max_count = -1
    for t in list(set(line_first_txt)):
        count = line_first_txt.count(t)
        if count > 1 and not re.match('[a-zA-Z가-힣0-9]', t):
            might_be_bullet.append((t,count))
            if count > max_count:
                might_be_product_bullet = t
                max_count = count
    return might_be_product_bullet, max_count

## 3️⃣ Script 뽑아오기